In [7]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

### C base no dataset acima apresentado e nos conceitos q adquiriu sobre Ensembles, explore a abordagem de Boosting através do treino de um modelo de GBM.

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.7+8-LTS-224, mixed mode, sharing)
  Starting server from C:\Users\Alice Dias\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ALICED~1\AppData\Local\Temp\tmpqb_58ztf
  JVM stdout: C:\Users\ALICED~1\AppData\Local\Temp\tmpqb_58ztf\h2o_Alice_Dias_started_from_python.out
  JVM stderr: C:\Users\ALICED~1\AppData\Local\Temp\tmpqb_58ztf\h2o_Alice_Dias_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Alice_Dias_d0aocp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.982 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


#### 1. Utilizando o H2O Flow e os critérios de config pra a generalização, treine o modelo avaliando-o c validação cruzada e um valor de n_fold=5.

In [5]:
# Carregamento dos dados
train = h2o.import_file("train.csv")
test = h2o.import_file("test.csv")

print(train.columns)

# Remover colunas irrelevantes
train = train.drop(["C1", "id"])
test = test.drop(["C1", "id"])

# Definir target e features
target = "satisfaction"
features = [col for col in train.columns if col != target]

# Converter target para categórica
train[target] = train[target].asfactor()

# Modelo GBM com validação cruzada
gbm_model = H2OGradientBoostingEstimator(
    ntrees=100,
    max_depth=5,
    learn_rate=0.1,
    nfolds=5,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=42
)

# Treinar o modelo
gbm_model.train(x=features, y=target, training_frame=train)

# Avaliação do modelo
performance_gbm = gbm_model.model_performance()
print(performance_gbm)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['C1', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes', 'satisfaction']
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.029529796630714207
RMSE: 0.17184235982642407
LogLoss: 0.10136171400767069
Mean Per-Class Error: 0.042193519739647314
AUC: 0.9939227785488678
AUCPR: 0.9928416431055977
Gini: 0.98784555709

##### a) Recorra aos conceitos sobre métricas de performance. Q análise faz ao modelo obtido?

O modelo GBM treinado c validação cruzada obteve resultados excelentes:

- AUC de 0.9939, mostrando alta capacidade de classificação binária.
- Baixo LogLoss (0.1014) e F1 Score alto (0.9533) garantem qualidade e equilíbrio entre precisão e recall.
- A confusion matrix está bem equilibrada, e o modelo tem uma baixa taxa de erro global (3.99%).

O modelo mostra uma clara capacidade de generalização, confirmada pela validação cruzada (nfolds=5) e pelas métricas estáveis e robustas.

##### b) Realize alterações aos critérios de config. Foi capaz de detetar mlhrias face ao modelo anterior?

In [6]:
# Modelo GBM com novos parâmetros
gbm_tuned = H2OGradientBoostingEstimator(
    ntrees=200,
    learn_rate=0.05,
    max_depth=7,
    nfolds=5,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=42
)

# Treinar o modelo
gbm_tuned.train(x=features, y=target, training_frame=train)

# Avaliação
performance_tuned = gbm_tuned.model_performance()
print(performance_tuned)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.02214499598583875
RMSE: 0.14881194839742792
LogLoss: 0.07608618946674395
Mean Per-Class Error: 0.030887380540516755
AUC: 0.9967588668161326
AUCPR: 0.9960898369574217
Gini: 0.9935177336322651

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.37057372088886864
                         neutral or dissatisfied    satisfied    Error    Rate
-----------------------  -------------------------  -----------  -------  -----------------
neutral or dissatisfied  57304                      1575         0.0267   (1575.0/58879.0)
satisfied                1577                       43448        0.035    (1577.0/45025.0)
Total                    58881                      45023        0.0303   (3152.0/103904.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
----------

Após alterar os critérios de config do modelo GBM, foram observadas mlhrias claras e significativas nas principais métricas de performance:

- AUC subiu de 0.9939 pra 0.9968, indicando uma mlhr capacidade de distinção entre as classes.
- LogLoss desceu de 0.1014 pra 0.0761, revelando previsões + confiantes.
- Acurácia aumentou pra 96.97%, c redução de erros.
- O modelo tornou-se + equilibrado, c < erro médio por classe (3.09%).

Esta melhoria deve-se à combinação de:

- mais árvores → mlhr aprendizagem ao longo do tempo.
- < taxa de aprendizagem → passos + pequenos e controlados.
- maior profundidade → árvores + expressivas.

#### 2. Utilizando Python, replique o processo de treino do modelo c os critérios de configuração q definiu no ponto anterior.

#### 3. Utilizando Python e c recurso à Grid Search, teste diferentes combinações de parâmetros (learn_rate, max_depth, sample_rate, etc.).

In [9]:
# Parâmetros a testar (pequeno número para execução rápida)
hyper_params = {
    "learn_rate": [0.1],
    "max_depth": [3, 5],
    "sample_rate": [0.8]
}

# Definição do modelo base com validação cruzada
gbm_grid = H2OGridSearch(
    model=H2OGradientBoostingEstimator(
        ntrees=100,
        nfolds=3,
        fold_assignment="Modulo",
        keep_cross_validation_predictions=True,
        seed=42
    ),
    hyper_params=hyper_params
)

# Treinar Grid Search
gbm_grid.train(x=features, y=target, training_frame=train)

# Ordenar por AUC
sorted_gbm_grid = gbm_grid.get_grid(sort_by="auc", decreasing=True)
print(sorted_gbm_grid)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
Hyper-Parameter Search Summary: ordered by decreasing auc
    learn_rate    max_depth    sample_rate    model_ids                                                       auc
--  ------------  -----------  -------------  --------------------------------------------------------------  --------
    0.1           5            0.8            Grid_GBM_py_2_sid_aca4_model_python_1744791199075_4768_model_2  0.993059
    0.1           3            0.8            Grid_GBM_py_2_sid_aca4_model_python_1744791199075_4768_model_1  0.987369


##### a) Q análise faz aos resultados?
- O modelo c trees + profundas (max_depth=5) obteve mlhr AUC doq c max_depth=3, oq mostra q o GBM beneficia de > expressividade nas trees, mm c - parâmetros.
- O valor de AUC = 0.9931 é excelente, e mt próximo do mlhr modelo GBM anterior (c + profundidade e + trees).
- Mm c ntrees = 100 e nfolds = 3, o desempenho foi mt competitivo → ótimo custo-benefício computacional.

##### b) Qual a combinação de configs + adequada? Oq considera cm validação pra esse processo?
🏆 Mlhr combinação:
- learn_rate = 0.1
- max_depth = 5
- sample_rate = 0.8

📋 Validação utilizada:
- Validação cruzada (nfolds = 3) c fold_assignment = "Modulo"
- Métrica principal: AUC
    - Escolhida por ser robusta mm c classes desbalanceadas
    - Mede a capacidade discriminatória do modelo independentemente do threshold